# Подсчёт сходимости (согласия) между кодировщиками в Python

In [2]:
import pandas as pd
import numpy as np
import json
import nltk
import krippendorff

In [15]:
df = pd.read_json("https://ihun.pythonanywhere.com/data/answers3.json")
df.head(10)
df.replace({"": np.nan, "Да": 2, "Нет": 0, "Не понятно": 1}, inplace=True)
num_names = [name for name in df.name.unique() if name.isdigit()]
spam = ["Иванов", "olessia", "Алена", "Вася", "тест", "лена", 'Воронова ', 'Устинова ', 'Анна Бушмелева '] + num_names
df = df[~df.name.isin(spam)]
names_step_1 = ['oandriyanova', 'ofadeeva', 'Бабкин Никита',
       'КутузоваВалентина', 'mariyapetrova', 'Малахова', 'Алена',
       'mnsergeeva1']
df = df[~df.name.isin(names_step_1)]

In [43]:
df.head()

,context,eth,isClear,isEthnonym,isRelated,isVerb,coder,raw_verb,textId,textNeg,verbNeg,verbNegContext,verbNegOwn
451,NaN,украинцев,2,2.0,1,2,18,происходит,5002691,5.0,NaN,0.0,NaN
452,NaN,басурманами,2,2.0,0,2,9,убивали,817777947,2.0,NaN,1.0,NaN
453,NaN,чеченец,2,2.0,2,2,14,приедет,612417281,3.0,3.0,0.0,NaN
454,NaN,чурки,2,2.0,2,2,10,едут,769791938,2.0,2.0,0.0,NaN
456,NaN,кавказца,2,2.0,2,2,0,ищет,4233490,1.0,4.0,0.0,NaN


In [17]:
df.rename(columns={"name": "coder"}, inplace=True)

In [36]:
df.coder = df.coder.astype('category').cat.codes

In [40]:
reliability_data = df.pivot(index="coder", columns="textId")["isClear"]

In [42]:
krippendorff.alpha(reliability_data, level_of_measurement="nominal")

0.0491371419641462

In [47]:
from nltk.metrics import agreement

In [48]:
taskdata = []

for coder, row in reliability_data.iterrows():
    for textId in row.index:
        taskdata.append((coder, str(textId), str(row.loc[textId])))
ratingtask = agreement.AnnotationTask(data=taskdata)
print("fleiss " + str(ratingtask.multi_kappa())) # Averages over observed and expected agreements for each coder pair
print("alpha Криппендорфа " + str(ratingtask.alpha()))
print("scotts " + str(ratingtask.pi()))
print()

fleiss 0.04915254237287492
alpha Криппендорфа 0.0491371419641462
scotts 0.04815484562316558

